$\newcommand{\xbf}{{\bf x}}
\newcommand{\ybf}{{\bf y}}
\newcommand{\wbf}{{\bf w}}
\newcommand{\Ibf}{\mathbf{I}}
\newcommand{\Xbf}{\mathbf{X}}
\newcommand{\Rbb}{\mathbb{R}}
\newcommand{\vec}[1]{\left[\begin{array}{c}#1\end{array}\right]}
$

# Classification multi-classe
Matériel de cours rédigé par Pascal Germain, 2019
************

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
import torch
from torch import nn
torch.__version__ # Ce notebook a été conçu avec la version '1.2.0' de pytorch

### L'ensemble de données «MNIST»

In [ ]:
repertoire_mnist = '../data/mnist/' # Modifier le répertoire au besoin

In [ ]:
def charger_mnist(repertoire, etiquettes=None, max_par_etiquettes=None):
    if etiquettes is None:
         etiquettes = np.arange(10)
    images_list = [None] * len(etiquettes)
    labels_list = [None] * len(etiquettes)
    for i, val in enumerate(etiquettes):
        nom_fichier = repertoire + f'mnist_{val}.gz'
        images_list[i] = np.genfromtxt(nom_fichier, max_rows=max_par_etiquettes, dtype=np.float32)
        nb = images_list[i].shape[0]

        labels_list[i] = i*np.ones(nb, dtype=np.int64)
        print(val, ':', nb, 'images')
        
    x = np.vstack(images_list)
    y = np.concatenate(labels_list)
    print('Total :', len(y), 'images')
    return x, y

In [ ]:
data_x, data_y = charger_mnist(repertoire_mnist, etiquettes=None, max_par_etiquettes=1000)
data_x = data_x / 255

In [ ]:
print('data_x:', data_x.shape)
print('data_y:', data_y.shape)

In [ ]:
plt.figure(figsize=(15,4))
plt.imshow(data_x, cmap=plt.cm.gray, aspect=.025)
plt.colorbar()

In [ ]:
exemple_index = 5002
exemple = data_x[exemple_index,:]
plt.figure(figsize=(5,4))
plt.imshow(exemple.reshape(28,-1), cmap=plt.cm.gray, aspect=1)
plt.colorbar()

### Quelques outils de la librairie `scikit-learn`

La librairie python `scikit-learn` (https://scikit-learn.org) contient une collection d'outils pour l'apprentissage automatique (machine learning), dont plusieurs algorithmes d'apprentissage «classiques» (régression logistique, SVM, forêts aléatoires, boosting, etc.) 

Dans ce TD et le suivant, nous utiliserons principalement ses fonctionnalités de base pour le traitement des données. Pour en apprendre davantage sur les fonctionnalités de `scikit-learn`, vous êtes invité à consulter le notebook optionnel nommé «`Extra - La librairie scikit-learn.ipynb`».

Pour tout connaître de scikit-learn, consultez le guide de l'utilisateur: https://scikit-learn.org/stable/user_guide.html

##### Séparation des données en un ensemble d'apprentissage et un ensemble de test

Voir: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=0.5, random_state=42)
print('train_x:', train_x.shape)
print('test_x:', test_x.shape)
print('train_y:', train_y.shape)
print('test_y:', test_y.shape)

##### Algorithme d'apprentissage (Exemple de la régression logistique)

Voir: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
from sklearn.linear_model import LogisticRegression

**Étape 1:** Initialiser l'algorithme d'apprentissage (constructeur de la *classe*).

In [ ]:
LogisticRegression?

In [ ]:
predicteur = LogisticRegression(C=1.)

**Étape 2:** Exécuter l'algorithme sur les données d'apprentissage (méthode `fit`).

In [ ]:
predicteur.fit?

In [ ]:
predicteur.fit(train_x, train_y)

**Étape 3:** Prédire sur des nouvelles données (méthode `predict`).

In [ ]:
predicteur.predict?

In [ ]:
predictions = predicteur.predict(test_x[0:100,:])
predictions

##### Évaluer la performance d'un algorithme d'apprentissage

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score?

In [ ]:
train_pred = predicteur.predict(train_x)
test_pred = predicteur.predict(test_x)
print('Précision train:', accuracy_score(train_y, train_pred) )
print('Précision test :', accuracy_score(test_y, test_pred))

*Regardons quelques erreurs de classifications*

In [ ]:
echecs = np.nonzero(predicteur.predict(test_x[0:100,:]) != test_y[0:100])
echecs

In [ ]:
fig, axes = plt.subplots(1, len(echecs[0]), figsize=(15, 4))
for i, ax in zip(echecs[0], axes):
    ax.imshow(test_x[i].reshape(28,28), cmap=plt.cm.gray)

# Réseaux de neurones multi-classe

#### Apprentissage par «minibatch»

**Notez bien:** L'apprentissage par «minibatch» n'est pas particulier aux réseaux multi-classe, mais nous en profitons pour expliquer comment l'utiliser dans *pytorch* au passage.

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
X = (torch.arange(1, 11, dtype=torch.float32) * torch.ones(3,10, dtype=torch.float32)).transpose(0,1)
X

In [ ]:
Y = 10 * torch.arange(1, 11) 
Y

In [ ]:
data = TensorDataset(X,Y)

In [ ]:
data[2]

In [ ]:
data[2:5]

In [ ]:
for x,y in data:
    print(x, '<-->', y)

In [ ]:
DataLoader?

In [ ]:
sampler = DataLoader(data, batch_size=3)
for t in range(4):
    print('******* EPOQUE', t)
    for x, y in sampler:
        print('---------------')
        print(x, '<-->', y)

In [ ]:
sampler = DataLoader(data, batch_size=3, shuffle=True)
for t in range(4):
    print('******* EPOQUE', t)
    for x, y in sampler:
        print('---------------')
        print(x, '<-->', y)

####  Couche de sortie «Softmax»

Étant donné un problème de classification à $C$ classes, la couche de sortie du réseau possédera $C$ neurones avec la fonction d'activation «Softmax»:

$$\text{Softmax}(a_{i}) = \frac{\exp(a_i)}{\sum_{j=1}^C \exp(a_j)}$$

In [ ]:
nn.Softmax?

In [ ]:
sm = nn.Softmax(dim=1) # Il faut spécifier la dimension selon laquelle appliquer la normalisation
sm(X)

Par la défition de la fonction SoftMax, le vecteur de sortie du réseau forme une distribution de probabilité:
$$\sum_{j=1}^C \text{Softmax}(a_{i}) = 1$$

In [ ]:
torch.sum(sm(X), dim=1)

In [ ]:
X2 = torch.randn(10, 3)
X2

In [ ]:
sm(X2)

In [ ]:
torch.sum(sm(X), dim=1)

####  Couche de sortie «LogSoftmax»

Pour des raisons de stabilité numérique, *pyTorch* préfère travailler avec le logarithme de l'activation SoftMax:

$$\text{LogSoftmax}(a_{i}) = \log\left(\frac{\exp(a_i) }{ \sum_j \exp(a_j)} \right)
= a_i - \log\left(\sum_j \exp(a_j) \right)$$

In [ ]:
nn.LogSoftmax?

In [ ]:
logsm = nn.LogSoftmax(dim=0) # Il faut spécifier la dimension selon laquelle appliquer la normalisation

In [ ]:
logsm(X2)

In [ ]:
torch.log(sm(X2))

In [ ]:
model = nn.Sequential(
                nn.Linear(28**2, 10),
                nn.LogSoftmax(dim=1) # Normalise chaque ligne
                )

In [ ]:
extrait_train_x = torch.tensor(train_x[0:5,:], dtype=torch.float32)
extrait_train_x.shape

In [ ]:
pred = model(extrait_train_x)
pred

$$\text{Softmax}(a_{i}) = \exp\Big(\text{LogSoftmax}(a_{i})\Big)$$

In [ ]:
torch.exp(pred)

In [ ]:
torch.exp(pred).sum(dim=1)

Le réseau prédit la classe correspondant à la valeur de sortie maximale:

$$\text{argmax}_i \bigg[\text{Softmax}(a_{i})\bigg] = \text{argmax}_i \bigg[\text{LogSoftmax}(a_{i})\bigg]$$

In [ ]:
pred.argmax(dim=1)

#### Perte du néfatif log vraissemblance associée à une sortie softmax

Pour nous faciliter la tâche, la fonction de perte `NLLLoss` est conçue pour gérer une paire d'arguments. Pour une minibatch de taille $m$ et un problème à $C$ classes, le calcul de la perte se fait à partir de:
1. La prédiction donnée par une activation `LogSoftmax` (de taille $m\times C$) ;
2. La sortie désirée sous la forme d'un vecteur de $m$ éléments, chacun de ces éléments étant un entier de $0$ à $C-1$.

In [ ]:
nn.NLLLoss?

In [ ]:
perte = nn.NLLLoss()

In [ ]:
Y = torch.ones(5, dtype=torch.int64)
perte(pred, Y)

## Enfin: Classifions MNIST avec un réseau de neurones

Le répertoire de ce TD contient un fichier `reseau_classif_generique.py`. Par ce fichier, nous vous fournissons le code *pytorch* qui vous aidera à effectuer l'apprentissage d'un modèle de réseau multi-classe.

La fonction d'apprentissage `ReseauClassifGenerique` utilise la descente de gradient stochastique avec momentum et «mini-batch». De plus, elle permet d'effectuer le «early stopping». 

In [ ]:
from reseau_classif_generique import ReseauClassifGenerique

In [ ]:
ReseauClassifGenerique?

In [ ]:
nb_entrees = 784
nb_sorties = 10
nb_neurones_cachees = 100

modele_plein = nn.Sequential(
            nn.Linear(nb_entrees, nb_neurones_cachees),
            nn.ReLU(),
            nn.Linear(nb_neurones_cachees, nb_sorties),
            nn.LogSoftmax(dim=1)
        )

reseau_mnist = ReseauClassifGenerique(modele_plein, eta=.1, alpha=.1, nb_epoques=100, patience=20)
reseau_mnist.apprentissage(train_x, train_y)

In [ ]:
train_pred = reseau_mnist.prediction(train_x)
test_pred = reseau_mnist.prediction(test_x)
print('Précision train:', accuracy_score(train_y, train_pred) )
print('Précision test :', accuracy_score(test_y, test_pred))

In [ ]:
plt.figure(figsize=(16,5))
stop_iter = reseau_mnist.meilleure_epoque-1
plt.plot(reseau_mnist.liste_erreur_train, label="Erreur d'entrainement")
plt.plot(reseau_mnist.liste_erreur_valid, label='Erreur de validation')
plt.scatter(stop_iter, reseau_mnist.liste_erreur_valid[stop_iter], s=300, c='r', marker='*', label='Early stopping')
plt.legend();

In [ ]:
plt.figure(figsize=(16,5))
plt.plot(reseau_mnist.liste_objectif, '--k', label='Valeur perte')
plt.scatter(stop_iter, reseau_mnist.liste_objectif[stop_iter], s=300, c='r', marker='*', label='Early stopping')
plt.legend();

-----------------

## À vous de jouer!



1. Répétez l'apprentissage du réseau de neurones avec différentes architectures (modèle): variez le nombre de couches, ainsi que le nombre de neurones par couches. 
2. Pour une architecture fixe, variez les paramètres de descente de gradient (eta et alpha), et analysez l'influence sur l'erreur de prédiction et la minimisation de la fonction objectif. Que se passe-t-il si les valeurs de ces paramètres sont trop petites ou trop grandes?
3. Ouvrez le fichier `reseau_classif_generique.py` dans un éditeur texte. Regardez le code pour comprendre son fonctionnement (en particulier l'optimisation par minibatch, le early stopping, et la méthode `prediction`).